In [ ]:
import tensorflow as tf
import pathlib
import random
import math
import numpy as np
from PIL import Image

In [ ]:
data_dir = pathlib.Path(r'../input/utkface-new')
data_dir

In [ ]:
train_raw = []
cv_raw = []
# test_raw = []
for i in range(1, 100):
    image_paths = list(map(lambda x: str(x), data_dir.glob(f'*/{i}_*.jpg')))
    class_label = (i - 1) // 5
    if image_paths:
        random.shuffle(image_paths)
        train_size = math.floor(len(image_paths) * 0.6)
        cv_size = math.floor(len(image_paths) * 0.2)
        train_raw.extend([(p, class_label) for p in image_paths[:train_size]])
        cv_raw.extend([(p, class_label) for p in image_paths[train_size:train_size + cv_size]])
        #test_paths.extend(image_paths[train_size + cv_size:])
random.shuffle(train_raw)
random.shuffle(cv_raw)
#random.shuffle(test_raw)

In [ ]:
train_raw

In [ ]:
train_paths, train_labels = zip(*train_raw)
train_paths = list(train_paths)
train_labels = list(train_labels)

In [ ]:
cv_paths, cv_labels = zip(*cv_raw)
cv_paths = list(cv_paths)
cv_labels = list(cv_labels)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))

In [ ]:
cv_data = tf.data.Dataset.from_tensor_slices((cv_paths, cv_labels))

In [ ]:
def read_image(image_path, label):
    image = tf.expand_dims(tf.io.decode_image(tf.io.read_file(image_path), dtype = tf.float32), axis = 0)
    #labels = tf.reshape(tf.one_hot(label, 20, axis = 0), (1, 20))
    labels = [label]
    return image, labels

In [ ]:
train_data = train_data.map(read_image)

In [ ]:
cv_data = cv_data.map(read_image)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 3, activation = 'relu', input_shape = (200, 200, 3)))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(128, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(256, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(256, activation = 'relu'))
model.add(tf.keras.layers.Dense(20))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', 
              loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True), 
              metrics = ['accuracy'])

In [ ]:
model.fit(train_data, epochs = 3)